In [2]:
import cv2
import torch 
import easyocr
from ultralytics import YOLO

ModuleNotFoundError: No module named 'torch'

In [ ]:
def detect_and_ocr(image_path, yolo_model_path, ocr_languages=['en']):
    # Load YOLO model
    model = YOLO(yolo_model_path)
    
    # Load image
    image = cv2.imread(image_path)
    
    # Perform object detection
    results = model(image)
    
    # Initialize OCR reader
    reader = easyocr.Reader(ocr_languages)
    
    # Process detected objects
    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Crop detected object
            cropped_obj = image[y1:y2, x1:x2]
            
            # Perform OCR on cropped object
            ocr_results = reader.readtext(cropped_obj)
            
            # Draw bounding box and detected text
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            for (bbox, text, score) in ocr_results:
                print(f"Detected Text: {text} (Score: {score})")
                cv2.putText(image, text, (x1, y1-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Save result image
    cv2.imwrite('detected_image.jpg', image)
    return image

# Example usage
detect_and_ocr(
    image_path='your_annotated_image.jpg', 
    yolo_model_path='yolov8n.pt'
)